In [ ]:
import pandas as pd
import os
import json

In [ ]:
success_df = pd.DataFrame(columns = ['Agent', 'Trajectory', 'Time'])

In [ ]:
directory = '/common/users/ac1771/imitation-game/results/agent_1/'

# iterate through all result json files
for file_name in os.listdir(directory):
    f = open(directory + file_name)

    # account for json coding errs
    try:
        result = json.load(f)

        if result['agent_1_nn']['success'] and result['agent_1_cnn']['success']:
            # first load agent 1 results
            success_df = success_df.append([{'Agent': '1_norm', 'Trajectory': result['agent_1']['trajectory_length'], 'Time': result['agent_1']['completion_time']}])
            
            # now neural network results
            success_df = success_df.append([{'Agent': '1_nn', 'Trajectory': result['agent_1_nn']['trajectory_length'], 'Time': result['agent_1_nn']['completion_time']}])
            success_df = success_df.append([{'Agent': '1_cnn', 'Trajectory': result['agent_1_cnn']['trajectory_length'], 'Time': result['agent_1_cnn']['completion_time']}])
    
    except ValueError:
        print(file_name)
    
    # Close file socket
    f.close()

success_df = success_df.reset_index()
success_df.head()

In [ ]:
success_df = success_df.convert_dtypes()
grouped_df = success_df.groupby(['Agent'])
avg_results_df = grouped_df.mean()
avg_results_df.head()